## Lab 04 Vector databases

### Creating db_url

In [1]:
from sqlalchemy.engine import URL

db_url = URL.create(
    drivername="postgresql+psycopg",
    username="postgres",
    password="password",
    host="localhost",
    port=5432,
    database="similarity_search_service_db",
)

### Models

In [7]:
from pgvector.sqlalchemy import Vector
from sqlalchemy import Integer, String
from typing import List
from sqlalchemy.orm import DeclarativeBase, Mapped, mapped_column


class Base(DeclarativeBase):
    __abstract__ = True


class Image(Base):
    __tablename__ = "images"
    VECTOR_LENGTH = 512

    id: Mapped[int] = mapped_column(Integer, primary_key=True)
    image_path: Mapped[str] = mapped_column(String(256))
    image_embedding: Mapped[List[float]] = mapped_column(Vector(VECTOR_LENGTH))


### Connecting to db

In [5]:
from sqlalchemy import create_engine

engine = create_engine(db_url)

### Creating Table

In [6]:
Base.metadata.create_all(engine)

### Queries

In [34]:
from sqlalchemy.orm import Session
import sqlalchemy
import numpy as np


def insert_image(
    engine: sqlalchemy.Engine, image_path: str, image_embedding: list[float]
):
    with Session(engine) as session:
        image = Image(image_path=image_path, image_embedding=image_embedding)
        session.add(image)
        session.commit()


N = 100
for i in range(N):
    image_path = f"image_{i}.jpg"
    image_embedding = np.random.rand(512).tolist()
    insert_image(engine, image_path, image_embedding)

with Session(engine) as session:
    image = session.query(Image).first()


def find_k_images(
    engine: sqlalchemy.Engine, k: int, orginal_image: Image
) -> list[Image]:
    with Session(engine) as session:
        # execution_options={"prebuffer_rows": True} is used to prebuffer the rows, this is useful when we want to fetch the rows in chunks and return them after session is closed
        result = session.execute(
            sqlalchemy.select(Image)
            .order_by(
                Image.image_embedding.cosine_distance(orginal_image.image_embedding)
            )
            .limit(k),
            execution_options={"prebuffer_rows": True},
        )
        return result


k = 10
similar_images = find_k_images(engine, k, image)

In [35]:
list(similar_images)[0][0].image_path

'image_0.jpg'

### Filtering

In [36]:
from sqlalchemy import select


def find_images_with_similarity_score_greater_than(
    engine: sqlalchemy.Engine, similarity_score: float, orginal_image: Image
) -> list[Image]:
    with Session(engine) as session:
        result = session.execute(
            select(Image).filter(
                Image.image_embedding.cosine_distance(orginal_image.image_embedding)
                > similarity_score
            ),
            execution_options={"prebuffer_rows": True},
        )
        return result

In [37]:
img = Image(image_path="test_path.jpg", image_embedding=np.random.rand(512).tolist())

find_images_with_similarity_score_greater_than(
    engine, similarity_score=0.65, orginal_image=img
)

## Steam

In [38]:
from datasets import load_dataset

dataset = load_dataset("FronkonGames/steam-games-dataset")

columns = dataset["train"].features
print(columns)

columns_to_keep = [
    "Name",
    "Windows",
    "Linux",
    "Mac",
    "About the game",
    "Supported languages",
    "Price",
]

N = 40000
dataset = dataset["train"].select_columns(columns_to_keep).select(range(N))


README.md: 0.00B [00:00, ?B/s]

data/train-00000-of-00001-e2ed184370a069(…):   0%|          | 0.00/123M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/83560 [00:00<?, ? examples/s]

{'AppID': Value('int64'), 'Name': Value('string'), 'Release date': Value('string'), 'Estimated owners': Value('string'), 'Peak CCU': Value('int64'), 'Required age': Value('int64'), 'Price': Value('float64'), 'DLC count': Value('int64'), 'About the game': Value('string'), 'Supported languages': Value('string'), 'Full audio languages': Value('string'), 'Reviews': Value('string'), 'Header image': Value('string'), 'Website': Value('string'), 'Support url': Value('string'), 'Support email': Value('string'), 'Windows': Value('bool'), 'Mac': Value('bool'), 'Linux': Value('bool'), 'Metacritic score': Value('int64'), 'Metacritic url': Value('string'), 'User score': Value('int64'), 'Positive': Value('int64'), 'Negative': Value('int64'), 'Score rank': Value('float64'), 'Achievements': Value('int64'), 'Recommendations': Value('int64'), 'Notes': Value('string'), 'Average playtime forever': Value('int64'), 'Average playtime two weeks': Value('int64'), 'Median playtime forever': Value('int64'), 'Medi

In [39]:
from sqlalchemy import Integer, Float, Boolean


class Games(Base):
    __tablename__ = "games"
    __table_args__ = {"extend_existing": True}

    # the vector size produced by the model taken from documentation https://huggingface.co/sentence-transformers/distiluse-base-multilingual-cased-v2
    VECTOR_LENGTH = 512  # check the model output dimensionality

    id: Mapped[int] = mapped_column(Integer, primary_key=True)
    name: Mapped[str] = mapped_column(String(256))
    description: Mapped[str] = mapped_column(String(4096))
    windows: Mapped[bool] = mapped_column(Boolean)
    linux: Mapped[bool] = mapped_column(Boolean)
    mac: Mapped[bool] = mapped_column(Boolean)
    price: Mapped[float] = mapped_column(Float)
    game_description_embedding: Mapped[List[float]] = mapped_column(
        Vector(VECTOR_LENGTH)
    )


Base.metadata.drop_all(engine)
Base.metadata.create_all(engine)

### Sentence transformers

In [41]:
from sentence_transformers import SentenceTransformer


checkpoint = "distiluse-base-multilingual-cased-v2"
model = SentenceTransformer(checkpoint, device="cpu")


def generate_embeddings(text: str) -> list[float]:
    return model.encode(text)

In [42]:
from tqdm import tqdm


def insert_games(engine, dataset):
    with tqdm(total=len(dataset)) as pbar:
        for i, game in enumerate(dataset):
            game_description = game["About the game"] or ""
            game_embedding = generate_embeddings(game_description)
            name, windows, linux, mac, price = (
                game["Name"],
                game["Windows"],
                game["Linux"],
                game["Mac"],
                game["Price"],
            )
            if name and windows and linux and mac and price and game_description:
                game = Games(
                    name=game["Name"],
                    description=game_description[0:4096],
                    windows=game["Windows"],
                    linux=game["Linux"],
                    mac=game["Mac"],
                    price=game["Price"],
                    game_description_embedding=game_embedding,
                )
                with Session(engine) as session:
                    session.add(game)
                    session.commit()
            pbar.update(1)

In [ ]:
insert_games(engine, dataset)  # TODO

  5%|████▏                                                                                  | 1944/40000 [02:38<51:34, 12.30it/s]


KeyboardInterrupt: 